In [1]:
import cv2
import os
import glob
import argparse
import pymc3 as pm
import numpy as np
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split

import theano 
import theano.tensor as T

Using TensorFlow backend.


In [2]:
def image_to_feature_vector(image, size=(32, 32)): return cv2.resize(image, size).flatten()

In [3]:
cats = map(image_to_feature_vector, [cv2.imread(cat) for cat in glob.glob('data/cats/*')])
dogs = map(image_to_feature_vector, [cv2.imread(dog) for dog in glob.glob('data/dogs/*')])

data = np.vstack((cats,dogs))
labels = np.array([1]*len(cats) + [0]*len(dogs))
data = np.array(data) / 255.0

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=.5)
ann_input = theano.shared(data)
ann_output = theano.shared(labels)

init_1 = np.random.randn(data.shape[1], 768)
init_2 = np.random.randn(768, 384)
init_out = np.random.randn(384)

In [ ]:
with pm.Model() as neural_network:
    # Weights from input to hidden layer
    W_in_1 = pm.Normal('w_in_1', 0, sd=1, 
                             shape=(data.shape[1], 768), 
                             testval=init_1)
    
    # Weights from 1st to 2nd layer
    W_1_2 = pm.Normal('w_1_2', 0, sd=1, 
                            shape=(768, 384), 
                            testval=init_2)
    
    # Weights from hidden lay2er to output
    W_2_out = pm.Normal('w_2_out', 0, sd=1, shape=(384,), testval=init_out)
    
    # Build neural-network using tanh activation function
    act_1 = T.nnet.relu(T.dot(ann_input, W_in_1))
    act_2 = T.nnet.relu(T.dot(act_1, W_1_2))
    act_out = T.nnet.sigmoid(T.dot(act_2, W_2_out))
    
    # Binary classification -> Bernoulli likelihood
    out = pm.Bernoulli('out', act_out, observed=ann_output)


/usr/local/lib/python2.7/site-packages/theano/tensor/basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


In [ ]:
from pymc3 import NUTS, sample
from scipy import optimize
from pymc3 import Slice

from pymc3 import find_MAP

map_estimate = find_MAP(model=neural_network)

print(map_estimate)

"""
with neural_network:
    # obtain starting values via MAP
    start = find_MAP(fmin=optimize.fmin_powell)

    # instantiate sampler
    step = Slice(vars=[sigma])

    # draw 5000 posterior samples
    trace = sample(5000, step=step, start=start)

"""